In [1]:
import json
import pandas as pd
import numpy  as np
from os import listdir
from os.path import isfile, join
import pathlib
import matplotlib.pyplot as plt

from datetime import datetime
import time
import requests

In [2]:
def gather_data(symbol, data_path, granularity, num_requests, start_date, last_date):
    
    current_date = start_date
    max_ite = (last_date - start_date)/granularity + 1
    ite = 0

    first_ite = True
    overwrite = True
    
    empty_data = True

    start_time = time.time()

    while current_date < last_date and ite < max_ite:

        end_date    = current_date + granularity * (num_requests)


        url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity={granularity}&start={current_date}&end={end_date}'
        response = requests.get(url)
        if response.status_code == 200:  # check to make sure the response from server is good
            data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
            data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
            data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
            data = data.set_index('date').sort_index()
        # if we failed to get any data, print an error...otherwise write the file
            if data is None:
                print("Did not return any data from Coinbase for this symbol")
                break
            elif not data.empty:
                if first_ite and overwrite:
                    data.to_csv(data_path)
                    first_ite = False
                    empty_data = False
                else:
                    dataset = pd.read_csv(data_path, index_col = 0)
                    dataset = pd.concat([dataset, data])
                    dataset.index = pd.to_datetime(dataset.index)
                    dataset.to_csv(data_path)
        else:
            print("Did not receieve OK response from Coinbase API")
            break

        current_date = end_date + granularity

        ite += 1

        if ite % 100 == 0:
            current_time = time.time()
            time_elapsed = current_time - start_time
            time_per_ite = time_elapsed/ite
            print(f'time left = {(max_ite - ite) * time_per_ite / 3600} hours')
            
    return empty_data

In [5]:
data_usd

[{'id': 'SHPING-USD',
  'base_currency': 'SHPING',
  'quote_currency': 'USD',
  'quote_increment': '0.000001',
  'base_increment': '1',
  'display_name': 'SHPING-USD',
  'min_market_funds': '1',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'status': 'online',
  'status_message': '',
  'trading_disabled': False,
  'fx_stablecoin': False,
  'max_slippage_percentage': '0.03000000',
  'auction_mode': False,
  'high_bid_limit_percentage': ''},
 {'id': 'XRP-USD',
  'base_currency': 'XRP',
  'quote_currency': 'USD',
  'quote_increment': '0.0001',
  'base_increment': '0.000001',
  'display_name': 'XRP-USD',
  'min_market_funds': '1',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'status': 'online',
  'status_message': '',
  'trading_disabled': False,
  'fx_stablecoin': False,
  'max_slippage_percentage': '0.05000000',
  'auction_mode': False,
  'high_bid_limit_percentage': ''},
 {'id': 'REN

In [3]:
f = open('coinbase_currencies.json')
data = json.load(f)
f.close()

data_usd = []

for dat in data:
    if dat['quote_currency'] == 'USD':
        data_usd.append(dat.copy())

tickers = []

for dat in data_usd:
    tickers.append(dat['display_name'])

In [4]:
granularity = int(60 * 60 * 24) 
num_requests = 300

# start_date   = 1672531200 # first jan 2023
start_date   = 1388530800 # frist jan 2014
last_date    = 1704063600

iteration = 1
time_series_recovered = 0

for symbol in tickers:
    data_path  = f"data_coinbase/{symbol}.csv"
    empty_data = gather_data(symbol, data_path, granularity, num_requests, start_date, last_date)
    
    if not empty_data:
        time_series_recovered += 1
    else:
        print('This time series was not recovered')
    
    print(iteration)
    print(f'Number of time series stored: {time_series_recovered}')
    print(symbol)
    print('='*70)
    
    iteration += 1

1
Number of time series stored: 1
SHPING-USD
2
Number of time series stored: 2
XRP-USD
3
Number of time series stored: 3
REN-USD
4
Number of time series stored: 4
RBN-USD
5
Number of time series stored: 5
MDT-USD
6
Number of time series stored: 6
BAT-USD
7
Number of time series stored: 7
TVK-USD
8
Number of time series stored: 8
ONDO-USD
9
Number of time series stored: 9
YFII-USD
10
Number of time series stored: 10
MINA-USD
11
Number of time series stored: 11
ILV-USD
12
Number of time series stored: 12
STX-USD
13
Number of time series stored: 13
SEI-USD
14
Number of time series stored: 14
BTC-USD
15
Number of time series stored: 15
TRB-USD
16
Number of time series stored: 16
EOS-USD
17
Number of time series stored: 17
SKL-USD
18
Number of time series stored: 18
PUNDIX-USD
19
Number of time series stored: 19
SUPER-USD
20
Number of time series stored: 20
REQ-USD
21
Number of time series stored: 21
API3-USD
22
Number of time series stored: 22
USDT-USD
23
Number of time series stored: 23
U